# 第十二章 计算机视觉

这一章，我们主要针对图神经网络在**计算机视觉**领域的应用进行介绍。图像或视频是一个非结构化的数据，当我们想要使用图对其进行处理的时候，我们需要先将其转化为结构化的图数据（包含顶点和边）。我们将通过两个任务：**图像分类**和**基于骨架的动作识别**，为大家介绍如何讲图神经网络应用在计算机视觉任务中。

## 12.1 计算机视觉背景
计算机视觉是计算机科学领域的一个分支，致力于使计算机能够模拟和理解人类视觉系统，从图像或视频中提取、分析和解释有意义的信息，以实现自动化的视觉感知和理解。

计算机使用数字表示图像。图像由像素组成，每个像素都包含一个数值，表示其在图像中的亮度或颜色信息。对于灰度图像，每个像素的数值表示灰度级别，通常在0到255之间。对于彩色图像，每个像素的数值由红、绿、蓝（RGB）三个分量的值组成，每个分量通常在0到255之间。这样，计算机通过一个二维矩阵或三维张量来存储和处理图像，其中每个元素对应图像中的一个像素。

## 12.2 图像分类
图像分类的目的是将图像分为不同的预定义类别或标签。通过图像分类，我们可以让计算机自动识别和归类图像，从而实现自动化的图像理解和组织。这对于许多应用非常重要，例如图像搜索、内容过滤、智能监控、医学图像分析等。通过准确的图像分类，计算机可以对大量的图像数据进行高效的处理和分析，从而提供更好的决策和洞察力。

### 12.2.1 图像的图结构表达
对于大小为 $H\times W\times 3$ 的图像，我们将其分为 $N$ 个块。通过将每个补丁转换为特征向量$\mathbf{x}_i\in\mathbb{R}^{D}$，我们有$X=[\mathbf{x}_1,\mathbf{x}_2,\cdots ,\mathbf{x}_N]$ 其中 $D$ 是特征维度，$i=1,2,\cdots,N$。这些特征可以被视为一组无序节点，表示为$\mathcal{V}=\{v_1,v_2,\cdots,v_N\}$。对于每个节点 $v_i$，我们找到其 $K$ 最近邻居 $\mathcal{N}(v_i)$ 并为所有 $v_j$ 添加一条从 $v_j$ 指向 $v_i$ 的边 $e_{ji}$ $\mathcal{N}(v_i)$。然后我们得到一个图 $\mathcal{G}=(\mathcal{V},\mathcal{E})$，其中$\mathcal{E}$表示所有边。我们将图的构建过程表示为 $\mathcal{G}=G(X)$。通过将图像视为图形数据，我们探索如何利用 GNN 提取其表示。

图表示图像的优点包括： 1）图是一种广义的数据结构，网格和序列可以看作图的特例； 2）图比网格或序列更灵活地对复杂对象进行建模，因为图像中的对象通常不是形状不规则的方形； 3）一个物体可以看作是多个部分的组合（例如，人可以大致分为头部、上半身、手臂和腿），图结构可以构建这些部分之间的联系； 4）GNN 的先进研究可以转移到解决视觉任务。

下图是图像的图结构表达。图像的网格、序列和图形表示的插图。在网格结构中，像素或块仅按空间位置排序。在序列结构中，2D 图像被变换为补丁序列。在图结构中，节点通过其内容链接起来，不受本地位置的约束。

<center>
    <img src="../../figures/12计算机视觉/image_as_graph.png" width=500>
    <br>
    <div>图12-1. 视觉图神经网络</div>
</center>

### 12.2.2 视觉图神经网络

视觉图神经网络（Vision Graph Neural Network，ViG）的结构如下图所示。

<center>
    <img src="../../figures/12计算机视觉/vig.png" width=500>
    <br>
    <div>图12-2. 视觉图神经网络</div>
</center>

一般来说，我们从特征 $X\in\mathbb{R}^{N\times D}$ 开始。我们首先根据特征构建一个图：$\mathcal{G}=G(X)$。图卷积层可以通过聚合邻居节点的特征来在节点之间交换信息。具体来说，图卷积的运算过程如下：
$$
\mathcal{G}' = {F}(\mathcal{G}, \mathcal{W}) = Update(Aggregate(\mathcal{G}, W_{agg}), W_{update}),
$$
其中 $W_{agg}$ 和 $W_{update}$ 分别是聚合和更新操作的可学习权重。更具体地说，聚合操作通过聚合邻居节点的特征来计算节点的表示，更新操作进一步合并聚合的特征：
$$
	\mathbf{x}'_i = h(\mathbf{x}_i, g(\mathbf{x}_i, \mathcal{N}(\mathbf{x}_i), W_{agg}), W_{update}),
$$
其中 $\mathcal{N}(\mathbf{x}_i^{l})$ 是 $\mathbf{x}_i^{l}$ 的邻居节点集合。这里我们采用最大相对图卷积，因为它的简单性和效率：
$$
	g(\cdot) = \mathbf{x}''_i = [\mathbf{x}_i,\max(\{\mathbf{x}_j-\mathbf{x}_i|j\in\mathcal{N}(\mathbf{x}_i)\}],
$$
$$
	h(\cdot) = \mathbf{x}'_i = \mathbf{x}''_iW_{update},
$$
其中偏置项被省略。上述图级处理可以表示为 $X'=\text{GraphConv}(X)$。

我们进一步介绍图卷积的多头更新操作。聚合特征 $\mathbf{x}''_i$ 首先被分成 $h$ 个头，即, $\textit{head}^1, \textit{head}^2, \cdots, \textit{head} ^h$，然后分别用不同的权重更新这些头。所有头都可以并行更新并连接起来作为最终值：
$$
	\mathbf{x}'_i = [\textit{head}^1W_{update}^1, \textit{head}^2W_{update}^2,\cdots, \textit{head}^hW_{update}^h].
$$
多头更新操作允许模型在多个表示子空间中更新信息，这有利于特征多样性。

### 12.2.3 视觉图神经网络代码

In [2]:
# updating soon

# Vision GNN: An Image is Worth Graph of Nodes

# https://github.com/huawei-noah/Efficient-AI-Backbones/tree/master/vig_pytorch

## 12.3 基于骨架的动作识别
基于骨架的动作识别是一种用于从人体骨架数据中识别和理解人的动作的方法。它通过使用传感器或深度摄像头等技术，捕捉人体关节点的位置和运动信息，然后利用机器学习或深度学习算法对这些数据进行分析和分类。基于骨架的动作识别广泛应用于人机交互、动作捕捉、运动分析和虚拟现实等领域。它可以实现实时的动作识别和动作控制，具有广泛的应用前景。

### 12.3.1 骨架的图结构表达
骨架序列通常由每帧中每个人体关节的 2D 或 3D 坐标表示。之前使用卷积进行骨骼动作识别的工作~\cite{Kim2017CVPRW} 连接所有关节的坐标向量以形成每帧的单个特征向量。
在我们的工作中，我们利用时空图来形成骨架序列的层次表示。
特别地，我们在具有$N$关节和$T$帧的骨架序列上构建无向时空图$G=(V,E)$，具有体内和帧间连接。

在此图中，节点集 $V = \{v_{ti} | t = 1,\ldots, T, i=1,\ldots,N\} $ 包括骨架序列中的所有关节。
作为 ST-GCN 的输入，节点 $F(v_{ti})$ 上的特征向量由帧 $t$ 上第 $i$ 个关节的坐标向量以及估计置信度组成。
%
我们分两步在骨架序列上构建时空图。
首先，根据人体结构的连通性，将一帧内的关节用边缘连接起来。
然后每个关节将连接到连续帧中的相同关节。
因此，该设置中的连接是自然定义的，无需手动分配零件。
这也使得网络架构能够处理具有不同数量的关节或关节连接的数据集。
例如，在 Kinetics 数据集上，我们使用 OpenPose~\cite{Cao2017Openpose} 工具箱输出 18 个关节的 2D 姿态估计结果，而在 NTU-RGB+D 数据集上，我们使用 3D 关节跟踪结果作为输入，产生 25 个关节。ST-GCN 可以在这两种情况下运行并提供一致的卓越性能。

正式地，边集$E$由两个子集组成，第一个子集描述了每一帧的骨架内连接，表示为 $E_S = \{v_{ti}v_{tj}| (i, j) \in H\}$，其中 $H$ 是自然连接的人体关节的集合。
第二个子集包含帧间边缘，它连接连续帧中的相同关节，如下所示 $E_F = \{v_{ti}v_{(t+1) i}\}$。
因此，$E_F$ 中某个特定关节 $i$ 的所有边都将代表其随时间变化的轨迹。

<center>
    <img src="../../figures/12计算机视觉/skeleton.png" width=500>
    <br>
    <div>图12-3. 骨架的图结构表达</div>
</center>

这些关键点可以通过openpose进行姿态估计获取，也可以手动标注。其数据维度一般为$（N, C, T, V, M ）$，其中：

$N$ 代表视频的数量，通常一个 batch 有 256 个视频（其实随便设置，最好是 2 的指数）；
$C$ 代表关节的特征，通常一个关节包含 $x,y,acc$ 等 3 个特征（如果是三维骨骼就是 4 个），$x,y$ 为节点关节的位置坐标，$acc$ 为置信度。
$T$ 代表关键帧的数量，一般一个视频有 150 帧。
$V$ 代表关节的数量，通常一个人标注 18 个关节。
$M$ 代表一帧中的人数，一般选择平均置信度最高的 2 个人。
需要注意$C$（特征），$T$（时间），$V$（空间）。

事实上，上述输入数据 $(N, C, T, V, M )$ 在输入至ST-GCN网络之前需要进行标准化操作。

该标准化是在时间维度上进行的，具体来说，就是标准化某节点在所有T个关键帧的特征值。其具体实现代码如下：

```python
# data normalization
N, C, T, V, M = x.size()
x = x.permute(0, 4, 3, 1, 2).contiguous()
x = x.view(N * M, V * C, T)
x = self.data_bn(x)
x = x.view(N, M, V, C, T)
x = x.permute(0, 1, 3, 4, 2).contiguous()
x = x.view(N * M, C, T, V)
其中函数data_bn定义如下：

self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
```

### 12.3.2 时空图卷积网络

时空图卷积网络 Spatial Temporal Graph Convolutional Network，ST-GCN

<center>
    <img src="../../figures/12计算机视觉/stgcn.png" width=500>
    <br>
    <div>图12-4. 时空图卷积网络</div>
</center>

**图划分策略**

在ST-GCN这篇文章中，作者的另一大创新点是通过对运动的分析引入了图划分策略，即建立多个反应不同运动状态（如静止，离心运动和向心运动）的邻接矩阵。作者在原文中提到其采用了三种不同的策略，分别为：

Uni-labeling，即与跟根节点相邻的所有结点具有相同的label。
Distance partitioning，即根节点本身的label设为0，其邻接点设置为1，如下图c所示。
Spatial configuration partitioning，是本文提出的图划分策略。也就是以根节点与重心的距离为基准（label=0），在所有邻接节点到重心距离中，小于基准值的视为向节心点（label=1），大于基准值的视为离心节点（label=2）。


其具体实现为
```python
A = []
for hop in valid_hop:
    a_root = np.zeros((self.num_node, self.num_node))
    a_close = np.zeros((self.num_node, self.num_node))
    a_further = np.zeros((self.num_node, self.num_node))
    for i in range(self.num_node):
        for j in range(self.num_node):
            if self.hop_dis[j, i] == hop:
                if self.hop_dis[j, self.center] == self.hop_dis[
                        i, self.center]:
                    a_root[j, i] = normalize_adjacency[j, i]
                elif self.hop_dis[j, self.
                                  center] > self.hop_dis[i, self.
                                                         center]:
                    a_close[j, i] = normalize_adjacency[j, i]
                else:
                    a_further[j, i] = normalize_adjacency[j, i]
    if hop == 0:
        A.append(a_root)
    else:
        A.append(a_root + a_close)
        A.append(a_further)
A = np.stack(A)
```

值得注意的是，hop类似于CNN中的kernel size。hop=0就是根节点自身，hop=1表示根节点与其距离等于1的邻接点们，也就是上图（a）的红色虚线框。

为了便于更好理解代码，我们默认上述两个循环中的
为根节点。因为条件if self.hop_dis[j, i] == hop限制，
可以视为根节点
的本身（hop=0）或者其邻接节点（hop=1）。

**网络结构**
其具体可以分为以下步骤：

步骤1：引入一个可学习的权重矩阵（与邻接矩阵等大小）与邻接矩阵按位相乘。该权重矩阵叫做“Learnable edge importance weight”，用来赋予邻接矩阵中重要边（节点）较大的权重且抑制非重要边（节点）的权重。
步骤2：将加权后的邻接矩阵A与输入X送至GCN中进行运算，实现空间维度信息的聚合。
步骤3：利用TCN网络（实际上是一种普通的CNN，在时间维度的kernel size>1）实现时间维度信息的聚合。
步骤4：引入了残差结构（一个CNN+BN）计算获得Res，与TCN的输出按位相加。

上述ST-GCN模块的代码实现如下：
```python
def forward(self, x, A):

    res = self.residual(x)
    x, A = self.gcn(x, A)
    x = self.tcn(x) + res

    return self.relu(x), A

# 其中残差结构self.residual定义如下：
self.residual = nn.Sequential(
    nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=1,
        stride=(stride, 1)),
    nn.BatchNorm2d(out_channels),
)

# GCN定义如下：
self.conv = nn.Conv2d(
        in_channels,
        out_channels * kernel_size,
        kernel_size=(t_kernel_size, 1),
        padding=(t_padding, 0),
        stride=(t_stride, 1),
        dilation=(t_dilation, 1),
        bias=bias)

def forward(self, x, A):
    assert A.size(0) == self.kernel_size

    x = self.conv(x)

    n, kc, t, v = x.size()
    x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
    x = torch.einsum('nkctv,kvw->nctw', (x, A))

    return x.contiguous(), A

#TCN定义如下
self.tcn = nn.Sequential(
    nn.BatchNorm2d(out_channels),
    nn.ReLU(inplace=True),
    nn.Conv2d(
        out_channels,
        out_channels,
        (kernel_size[0], 1),
        (stride, 1),
        padding,
    ),
    nn.BatchNorm2d(out_channels),
    nn.Dropout(dropout, inplace=True),
)
```


实际上，本文提出模通过不断堆叠ST-GCN从图结构输入中持续提取高级的语义特征，具体如下：

```python
self.st_gcn_networks = nn.ModuleList((
    st_gcn(in_channels, 64, kernel_size, 1, residual=False, **kwargs0),
    st_gcn(64, 64, kernel_size, 1, **kwargs),
    st_gcn(64, 64, kernel_size, 1, **kwargs),
    st_gcn(64, 64, kernel_size, 1, **kwargs),
    st_gcn(64, 128, kernel_size, 2, **kwargs),
    st_gcn(128, 128, kernel_size, 1, **kwargs),
    st_gcn(128, 128, kernel_size, 1, **kwargs),
    st_gcn(128, 256, kernel_size, 2, **kwargs),
    st_gcn(256, 256, kernel_size, 1, **kwargs),
    st_gcn(256, 256, kernel_size, 1, **kwargs),
))

# initialize parameters for edge importance weighting
if edge_importance_weighting:
    self.edge_importance = nn.ParameterList([
        nn.Parameter(torch.ones(self.A.size()))
        for i in self.st_gcn_networks
    ])
else:
    self.edge_importance = [1] * len(self.st_gcn_networks)

# ST-GCN与可学习的权重矩阵不断重复与堆叠
for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
 x, _ = gcn(x, self.A * importance)
之后，和一般的分类任务类似，作者引入了全局平均池化以及全卷积层输出预测分支，如下：

# global pooling
x = F.avg_pool2d(x, x.size()[2:])
x = x.view(N, M, -1, 1, 1).mean(dim=1)

# prediction
x = self.fcn(x)
x = x.view(x.size(0), -1)
```

### 12.3.3 时空图卷积网络基于骨架的的动作识别代码
下面我们将加载数据集，并进行网络的训练和评估

In [ ]:
#updating soon

## 12.4 参考资料
图深度学习从理论到实践 包勇军、朱小坤、颜伟鹏、姚普 清华大学出版社

[Vision GNN: An Image is Worth Graph of Nodes](https://arxiv.org/pdf/2206.00272.pdf)

[Spatial Temporal Graph Convolutional Networks for Skeleton-Based Action Recognition](https://arxiv.org/pdf/1801.07455.pdf)


[Action Detection ST-GCN论文与代码解析](https://zhuanlan.zhihu.com/p/418989078)